## QnA Assistant with Persistent Memory
- This is follow-up of Part 1 of simple QnA Bot
- Here we will be using Azure cosmosDB as memory to store conversation hostory.

In [1]:
# install azure cosmos package
%pip install azure-cosmos

     ---------------------------------------- 0.0/303.2 kB ? eta -:--:--
     ---- -------------------------------- 41.0/303.2 kB 653.6 kB/s eta 0:00:01
     ---------------------------------- --- 276.5/303.2 kB 2.8 MB/s eta 0:00:01
     -------------------------------------- 303.2/303.2 kB 2.7 MB/s eta 0:00:00
     ---------------------------------------- 0.0/211.8 kB ? eta -:--:--
     -------------------------------------- 211.8/211.8 kB 6.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


### Import Libraries

In [2]:
import os, uuid, json
from dotenv import load_dotenv
from azure.cosmos import CosmosClient
from openai import OpenAI

### Initialize Clients

In [3]:
load_dotenv('config.env')
# OpenAI Client
openai_api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=openai_api_key)


# CosmosDB Client
cosmosdb_endpoint = os.getenv("COSMOSDB_ENDPOINT")
cosmosdb_api_key = os.getenv("COSMOSDB_API_KEY")
cosmos_client = CosmosClient(cosmosdb_endpoint, cosmosdb_api_key)

# Establish connection to specific DB and container within it
database = cosmos_client.get_database_client('mydatabase')
container = database.get_container_client('conversation')

ServiceRequestError: Invalid URL '': No scheme supplied. Perhaps you meant https://?

### Database functions
Requirement
- CosmosDB Container will have items with - id (mandat unique id of all items), session_id, user_id, conversation_history, last_updated datetime
- Input will be user_id, session_id and message.
- If entry already exist for user_id and session_id, append new message in convo history. else create a new entry/item in cosmosDB.


In [4]:
from datetime import datetime, timezone
import uuid

In [5]:
# Fetch a specific session_id doc for user with user_id

def get_session_doc(user_id, session_id):
    query = f"SELECT * FROM c WHERE c.user_id = '{user_id}' and c.session_id = '{session_id}'"
    items = list(container.query_items(query=query, enable_cross_partition_query=True))
    return items[0] if items else None

In [ ]:
# create or update existing record conversation history
MAX_HISTORY_MESSAGES = 50

def save_conversation(user_id, session_id, role, content):

    # check if session exist in DB
    record = get_session_doc(user_id, session_id)

    # if session do not exist, create new item
    if not record:
        new_item = {
            "id": str(uuid.uuid4()),
            "user_id": user_id,
            "session_id": session_id,
            "conversation_history": [
                {"role":role, "content":content}
            ],
            "last_updated": datetime.now(timezone.utc).isoformat(timespec='milliseconds')
        }
        container.create_item(new_item)
    else:
        record['conversation_history'].append({"role":role, "content":content})

        # Trim to last N messages
        if len(record["conversation_history"]) > MAX_HISTORY_MESSAGES:
            record["conversation_history"] = record["conversation_history"][-MAX_HISTORY_MESSAGES:]
            
        record['last_updated'] = datetime.now(timezone.utc).isoformat(timespec='milliseconds')
        container.replace_item(item=record['id'], body=record)

In [8]:
# fetch conversation history

def fetch_convo_history(user_id, session_id):
    record = get_session_doc(user_id, session_id)
    if record:
        return record['conversation_history']
    else:
        return []

### OpenAI assistant
- With conversation history as context for response

In [9]:
def openai_response(user_id, session_id, system_prompt, human_prompt):
    # Load existing convo history
    conversation = fetch_convo_history(user_id, session_id)

    # Add system prompt if first interaction
    if not any(m["role"] == "system" for m in conversation):
        save_conversation(user_id, session_id, "system", system_prompt)
        conversation.append({"role": "system", "content": system_prompt})

    # Add new user message
    save_conversation(user_id, session_id, "user", human_prompt)
    conversation.append({"role": "user", "content": human_prompt})

    # Call OpenAI with full conversation
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=conversation
    )

    reply = response.choices[0].message.content.strip()

    # Save assistant reply
    save_conversation(user_id, session_id, "assistant", reply)

    return reply


### Test the Assistant

In [10]:
user_id = "test_user"
session_id = "session_1"
system_prompt = "You are a helpful QnA bot."
user_query = "What is the capital of India?"

In [ ]:
answer = openai_response(user_id, session_id, system_prompt, human_prompt=user_query)
print("Bot:", answer)